### UC Berkeley, MICS, W202-Cryptography
### Week 04 Breakout 4
### Diffie Hellman Man in the Middle Attack

As mentioned in the last breakout, Diffie Hellman predates RSA, and unlike RSA, is subject to Man in the Middle Attacks:

* The man in the middle attack occurs when an unauthorized 3rd party "Mallory", secretly comes between "Alice" and "Bob".  

* Alice thinks Mallory is Bob.  

* Bob thinks Mallory is Alice.  

* Obviously, if we had a secure identity authentication protocol of some sort, Alice could verify Bob's identity, and Bob could verify Alice's identity.  The man in the middle attack is only successful when there is no secure identity authentication protocol in place.

* Alice tries to execute a key exchange with Bob, but Mallory intercepts, executes a Diffie Hellman key exchange with Alice, and creates a key between Mallory and Alice.  

* Likewise, Bob tries to execute a key exchange with Alice, but Mallory intercepts, executes a Diffie Hellman key exchange with Bob and create a key between Mallory and Bob.

* When Alice sends a message to Bob, Mallory intercepts it, decrypts the message using his key with Alice, can now read the message in plaintext, then re-encrypts the message using his key with Bob, and sends it to Bob. 

* When Bob sends a message to Alice, Mallory intercepts it, decrypts the message using his key with Bob, can now read the message in plaintext, then re-encrypts the message using his key with Alice, and send it to Alice. 

* Messages go both ways between Alice and Bob, but Mallory is intercepting and reading (and possibly changing) them both ways.  Neither Bob nor Alice know that their messages are being intercepted and read by Mallory.


In [1]:
import hashlib
from sage.all import *

In [2]:
def my_print_number(label, x):
    "prints a number in decimal, number of digits, hex, number of bits"
    
    print ("\n", label, '\n')
    print ("decimal:", "{:,}".format(x), "\n")
    print ("number of digits:", x.ndigits(), "\n")
    print ("hex:", x.hex(), "\n")
    print ("number of bits:", x.nbits(), "\n")

In [3]:
# b is the number of bits in p
# we will use it to ensure that Alice's and Bob's exponents are of similar size to b

b = 1024

### p and g are publically known.  p is a large prime number of similar sizing as in RSA.  g is the smallest generator for p.  the security of Diffie Hellman does not depend on the size of g, so a small g is fine.

In [4]:
# p and g are publically known

# g is the smallest generator we found for p
# p is a large prime 

# it's always best to select a p in which 2 is a generator as it speeds things up without losing security

g = 2

p = 0xa42667765996150b81063e7a253b91b5da58af0ff49328bf8a3f9f38bdf1e357272af338846d1d612a71a31e35f590d6c131634306ed75bdb45c4cb9b15072127f2afac831560bc5c1f697479b1d7d6f0da191faa68eaabf5b138ea782f97ae6624c9244150d38f28231993fe0d6435d852d42782b8263aea061a40da802903b

In [5]:
my_print_number("p", p)


 p 

decimal: 115,270,061,416,921,866,062,188,383,030,847,280,859,479,926,375,604,672,663,124,358,614,555,215,978,993,787,838,406,298,575,203,649,480,844,189,752,685,847,453,468,731,960,555,557,838,447,956,623,493,236,754,000,427,739,727,368,609,894,359,124,610,032,258,486,582,384,311,834,472,633,840,703,883,057,700,772,340,022,758,927,012,248,883,130,603,620,091,710,872,172,650,710,125,452,082,076,641,079,289,492,705,339 

number of digits: 309 

hex: a42667765996150b81063e7a253b91b5da58af0ff49328bf8a3f9f38bdf1e357272af338846d1d612a71a31e35f590d6c131634306ed75bdb45c4cb9b15072127f2afac831560bc5c1f697479b1d7d6f0da191faa68eaabf5b138ea782f97ae6624c9244150d38f28231993fe0d6435d852d42782b8263aea061a40da802903b 

number of bits: 1024 



### Alice will get a random exponent of similar size to p, and can be even or odd, prime or composite.  Alice will raise the generator g to the power of her exponent in modulo p and send this value to Bob.  (However, Mallory is pretending to be Bob, so Alice really sends the message to Mallory)

In [6]:
def my_get_random_exponent(b, p):
    "given a number of bits, find a random exponent of that bit size -1"
    
    upper_limit = (p-2)
    lower_limit = ( (p-2) // 2)
    
    return Integer(randint(lower_limit, upper_limit))

In [7]:
alice_exponent = my_get_random_exponent(b, p)

In [8]:
my_print_number("Alice's exponent", alice_exponent)


 Alice's exponent 

decimal: 63,597,514,939,982,315,210,490,359,025,370,511,642,701,931,404,592,854,536,616,474,995,371,118,330,645,286,798,372,833,723,714,135,883,576,955,254,988,609,272,099,507,750,990,541,874,843,666,579,486,816,676,280,230,171,465,518,532,360,362,384,992,374,376,516,320,077,186,111,828,694,049,913,196,875,303,474,368,281,155,833,785,330,318,825,716,995,581,654,932,842,046,583,834,027,184,109,237,280,693,200,648,423 

number of digits: 308 

hex: 5a90dcb12b80944c235a1d12441abd430d6dedd1e10082fa7f4f2fb656d1431b3d758b3bfc7dfbbb2d8da20877f9b602436748ca556725393c85b0cc174a25910541c3517c90b8b9da12ff7a656c5918db0d96dc1b80f2fbc35d95e9a02519f60994717b3b016bf2bcf83a98e2ef63ecd33ec7c279e20395eaabea9e59dc78e7 

number of bits: 1023 



In [9]:
alice_sends = power_mod(g, alice_exponent, p)

In [10]:
my_print_number("Alice sends Bob (but it will really go to Mallory who is pretending to be Bob)", alice_sends)


 Alice sends Bob (but it will really go to Mallory who is pretending to be Bob) 

decimal: 50,890,254,711,574,206,731,335,034,738,553,834,242,912,602,517,981,727,627,099,412,249,668,807,883,836,792,889,590,048,656,529,904,060,508,703,172,132,515,986,572,166,805,017,855,743,402,769,573,206,976,655,228,328,093,320,612,881,362,784,864,402,586,018,823,645,684,482,095,493,481,467,227,904,789,239,486,334,445,674,341,114,974,634,213,118,428,376,370,958,590,783,106,145,133,267,562,612,113,331,910,090,366 

number of digits: 308 

hex: 48785a536c0c23937e581ac3e717dbd9dec4677d5b1d91a99e30ba76e4c1cd25fef3bbca2631a7ddb82b3503025708bb21d4e54c55fb8a64e253d17aac556b54d5f730e029e6ce5070e99c9b4a568d4fee0d19b3702fa028a92eff6b1ef54412b19056712e8fda044ed5cf25c6ddae2d64001bd96ff205e162a5e00246301e7e 

number of bits: 1023 



### Alice just sent her value to Mallory, thinking she was sending it to Bob. Mallory has now captured Alice's value.  Mallory will generate his own exponent, raise the generator to the exponent, and cut it modulo p.  Mallory will send this value to Bob. Bob will think he's receiving it from Alice.

In [11]:
mallory_exponent_4_bob = my_get_random_exponent(b, p)

In [12]:
my_print_number("Mallory's exponent for Bob (Bob thinks Mallory is Alice)", mallory_exponent_4_bob)


 Mallory's exponent for Bob (Bob thinks Mallory is Alice) 

decimal: 112,855,847,083,260,135,506,401,188,299,713,394,617,533,971,515,103,922,512,989,507,383,580,628,962,223,694,346,812,720,064,630,378,680,552,134,164,555,708,956,932,449,169,862,736,660,092,535,856,056,354,919,990,870,628,610,090,403,309,406,603,020,580,272,431,755,275,188,040,049,529,406,339,704,170,368,917,610,851,970,547,328,568,379,619,634,013,930,618,727,009,570,897,107,400,317,591,342,309,010,560,068,252 

number of digits: 309 

hex: a0b6499f2f64eff043ee16f90da9bedf592fe8016b3cdc2c38da9a8cc1cfc99c9006b46648ad728775476234d8d053ff05be8d2f6909ac052e897bd35d3f203cdc4ac73bea9e417c5b8b204e06df441a857e323dbf9813bfdcba87ff408cba129412fc062afbf778146404a10360bef9779b8aeeba7dc870c29a1ede25d52a9c 

number of bits: 1024 



In [13]:
mallory_sends_2_bob = power_mod(g, mallory_exponent_4_bob, p)

In [14]:
my_print_number("Mallory sends to Bob (Bob thinks Mallory is Alice)", mallory_sends_2_bob)


 Mallory sends to Bob (Bob thinks Mallory is Alice) 

decimal: 63,846,965,260,473,252,545,499,096,257,978,662,024,528,377,417,210,975,113,581,988,537,362,800,811,569,487,699,192,215,957,632,643,753,061,709,322,815,276,811,382,540,581,273,551,670,548,362,631,212,715,385,509,761,631,173,690,534,171,052,901,949,348,255,477,520,116,166,383,632,786,169,699,969,460,687,429,018,164,042,083,448,431,093,936,038,319,027,600,389,654,827,095,046,530,080,813,781,280,189,192,979,255 

number of digits: 308 

hex: 5aebccfba2356b34607c1586807ceb34c4226912631707d02f87ddbfe1337be1ba6a1bb7fe1be3f927eb30e8c7905db3830e6fe4408f436ec994de823090cf5e52fdb282e10532e53f9f38d231603fd3d086a34737db57b02f3864639179ff1c6b37b7f61caac07c6af2dcfe88a84f84256785276f5c4ef34613ee163f79c337 

number of bits: 1023 



### Likewise, Bob will get a random exponent of similar size to p, and can be even or odd, prime or composite.  Bob will raise the generator g to the power of his exponent in modulo p and send this value to Alice. (However, Mallory is pretending to be Alice, so Bob really sends the message to Mallory)

In [15]:
bob_exponent = my_get_random_exponent(b, p)

In [16]:
my_print_number("Bob's exponent", bob_exponent)


 Bob's exponent 

decimal: 57,961,264,725,450,499,937,215,258,484,770,366,019,378,173,883,985,407,453,116,957,522,818,738,019,433,572,697,009,675,768,608,185,923,975,036,981,087,607,202,301,988,645,031,250,083,964,811,496,722,444,454,946,054,485,874,886,237,335,197,128,418,296,320,375,637,354,775,311,162,465,446,706,399,923,913,683,015,552,533,165,105,909,051,911,469,682,201,064,809,264,998,017,140,415,507,146,627,704,144,916,030,926 

number of digits: 308 

hex: 528a21f70eceaa74cae2a61d4938d0a4a072e4d00f196f80c52470dd9975de404cc5d301d4175f550c5286ddcddc6e879d286480d9a25e8dad3691b320c46b6ce636b1baaaea962deb604818a41ae52eab5dfcba80232826eebf6f444adcbed8b3450d27b35b71e96b17631367eed4227df439359a70928499ef75fe92aaa5ce 

number of bits: 1023 



In [17]:
bob_sends = power_mod(g, bob_exponent, p)

In [18]:
my_print_number("Bob sends Alice (but it will really go to Mallory who is pretending to be Alice)", bob_sends)


 Bob sends Alice (but it will really go to Mallory who is pretending to be Alice) 

decimal: 35,650,786,387,484,314,423,445,353,978,893,321,863,291,646,871,737,784,590,420,196,569,694,393,725,069,454,820,040,544,414,833,205,155,808,418,428,827,076,257,913,973,552,371,200,266,730,943,734,882,165,841,837,872,119,726,609,778,976,610,255,618,942,539,551,763,889,265,646,491,143,161,223,663,433,956,863,352,732,354,677,621,776,754,658,124,142,107,494,233,464,238,477,416,771,571,588,663,282,852,777,307,263 

number of digits: 308 

hex: 32c4b630ce3c11e93c91f79aa82dc2944b5c72e1fdb59952fdc13a200664590e5145559af70f184b624628c199b2b8d59c7687fdc0a00f27febe6279f4be509f3b0a6255668fe4f5c11fd222d4581c3c92e34b4a7195bc7a9b2308b5a2fa0917081bea6865f0a5b0c2091d3b00b804aba231d953ed760d2c119d1423b4f62c7f 

number of bits: 1022 



### Bob just sent his value to Mallory, thinking he was sending it to Alice. Mallory has now captured Bob's value.  Mallory will generate his own exponent, raise the generator to the exponent, and cut it modulo p.  Mallory will send this value to Alice. Alice will think she's receiving it from Bob.

In [19]:
mallory_exponent_4_alice = my_get_random_exponent(b, p)

In [20]:
my_print_number("Mallory's exponent for Alice (Alice thinks Mallory is Bob)", mallory_exponent_4_alice)


 Mallory's exponent for Alice (Alice thinks Mallory is Bob) 

decimal: 65,877,594,299,354,536,310,110,205,669,821,193,496,232,072,567,476,890,754,410,581,873,863,706,658,740,960,793,402,923,175,089,636,004,095,286,777,407,503,812,187,598,359,271,539,716,910,973,864,242,803,662,722,127,456,474,566,723,815,213,584,843,438,219,039,055,031,381,383,406,527,293,463,108,735,079,294,787,190,636,437,856,104,332,331,916,172,208,487,508,063,427,423,625,839,516,223,293,323,860,834,430,522 

number of digits: 308 

hex: 5dd0143412b1ef065624fe0fdb89358ae52cf108a15f1309ed9f429b87f2660c890d4cfa2e4b76c12c60b5db0328c7f86243c8dd14f0a910c86a8934d73fbc8acf6c7e73e079626fd2fe1f2e01aee2b6776063c669fa1c80d21b31a76a7fcc3815ac74a7011a92ff54ae48ffbd1250b5ac9574cb1201e27b88b8b3fdaacaae3a 

number of bits: 1023 



In [21]:
mallory_sends_2_alice = power_mod(g, mallory_exponent_4_alice, p)

In [22]:
my_print_number("Mallory sends to Alice (Alice thinks Mallory is Bob)", mallory_sends_2_alice)


 Mallory sends to Alice (Alice thinks Mallory is Bob) 

decimal: 16,783,687,122,057,224,663,515,241,736,073,948,736,560,460,557,408,275,079,600,428,145,492,065,099,284,431,370,343,312,435,310,944,000,295,823,011,857,782,757,481,708,326,276,269,664,313,473,375,120,352,424,322,240,393,052,648,042,964,354,352,404,637,326,749,563,174,246,159,394,042,384,710,038,879,534,250,363,894,141,112,535,989,674,669,504,647,225,035,955,184,331,972,833,464,788,327,373,382,961,466,319,139 

number of digits: 308 

hex: 17e69875210f44d0d21ff3bfca53f79a7b06e5de5380efc9d20ba28c5850b0eed612c60a1274d41c6fef02d2e2ad518be1f8ea1791a6dc9ee68952c2231581c9765d6abd9c4a221ab7a8b1daa7e303ec879797b117e7ad2544a4c97561987ac5698c54d9e426426e1f67c5f46fe25ec001e4322e67509f9aa323599b0c9e2123 

number of bits: 1021 



### Alice will take the value sent to her by Mallory and raise it to the power of her exponent in modulo p. This will be the key. Alice thinks it's the key she shares with Bob, but it's actually the key she shares with Mallory

In [23]:
alice_key = power_mod(mallory_sends_2_alice, alice_exponent, p)

In [24]:
my_print_number("Alice's key (with Mallory!)", alice_key)


 Alice's key (with Mallory!) 

decimal: 68,754,265,980,195,706,270,707,095,100,515,106,088,826,729,104,650,078,712,452,830,368,406,844,338,019,567,221,490,478,417,412,531,568,337,419,355,530,977,469,212,954,460,396,764,062,820,885,688,419,977,369,765,102,424,890,699,428,806,320,601,993,154,145,719,691,547,716,588,619,932,542,662,702,880,656,812,158,779,691,224,668,917,866,025,961,906,764,181,633,425,077,283,109,957,512,847,694,481,930,168,386,598 

number of digits: 308 

hex: 61e8c981706113937bb031b44b5c335699894cc25fdf5d74892429b0f1ef40acf6c549c0e91165a1c24d1b560b0fde1ea98d020207f99a44a8e45117b4ae327023f60638964b333e03881539987647edf77f00562092bde54af6719de7158b78edf1e45f7fd094204cf19d5caf9d72aa8722fb72ad4aa20c9f750e0f47794426 

number of bits: 1023 



### Likewise, Bob will take the value sent to him by Mallory and raise it to the power of his exponent in modulo p. This will be the key.  Bob thinks it's the key he shares with Alice, but it's actually the key he shares with Mallory

In [25]:
bob_key = power_mod(mallory_sends_2_bob, bob_exponent, p)

In [26]:
my_print_number("Bob's key (with Mallory!)", bob_key)


 Bob's key (with Mallory!) 

decimal: 91,348,132,132,247,811,008,126,234,949,328,425,686,893,779,688,276,477,411,124,029,841,351,064,787,059,736,731,070,327,326,572,809,975,058,590,226,848,578,444,831,990,778,328,294,728,978,080,944,429,847,544,482,062,191,083,253,506,321,735,120,652,855,577,401,844,630,278,706,441,364,761,175,796,523,587,658,657,511,311,408,637,199,412,076,370,836,244,501,642,887,665,413,350,626,007,964,100,359,854,845,910,524 

number of digits: 308 

hex: 821584d56f03590099b8ff2f7c3e699b7791ac9c774c331053f0a01aa7b68f37a1b730524fe795beed6ef2abb061e5c1688e6d5f1fb6edff378a03637a5b3e362ef3649eeb5c93d6c2cf751b89b460367b30cd1816d212aa286ec9cbdd51978c88584b4f0bc132781953acac8c25359cd1b519af78f08ec8bde8cb5926bcddfc 

number of bits: 1024 



### Verify that Alice's key and Bob's key do NOT match, due to the man in the middle attack.

In [27]:
if alice_key == bob_key:
    print ("Alice's key matches Bob's key!")
else:
    print ("Alice's key does NOT match Bob's key due to the man in the middle attack!")

Alice's key does NOT match Bob's key due to the man in the middle attack!

### Verify that Mallory can generate a key to match Alice's key

In [28]:
mallory_key_alice = power_mod(alice_sends, mallory_exponent_4_alice, p)

if alice_key == mallory_key_alice:
    print ("Mallory's key matches Alice's key!")
else:
    print ("Error - Mallory's key does NOT match Alice's key!")

Mallory's key matches Alice's key!


### Verify that Mallory can generate a key to match Bob's key

In [29]:
mallory_key_bob = power_mod(bob_sends, mallory_exponent_4_bob, p)

if bob_key == mallory_key_bob:
    print ("Mallory's key matches Bob's key!")
else:
    print ("Error - Mallory's key does NOT match Bob's key!")

Mallory's key matches Bob's key!


### Now that Mallory has keys for both Bob and Alice he can successfully intercept, decrypt, read, possibly change, and re-encrypt all messages to and from Alice and Bob